<a href="https://colab.research.google.com/github/jrmaza/machine-learning/blob/main/Electiva03_rdkit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Módulo de predicción de solubilidad utilizando [RDKit](https://www.rdkit.org/)**

## [RDKit](https://www.rdkit.org/)
La colección de librerías y módulos para aplicaciones de bio/quimioinformática y aprendizaje automatizado en python es RDKit. De momento, la instalación debe realizarse por medio del gestor anaconda

### **1. Instalación de RDKit a través de conda**
(~1min)

In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

### **2. Conjunto de datos de solubilidad de Delaney**
En 2004, el autor John S. Delaney publicó una investigación para la estimación de la solubilidad acuosa de sustancias a partir de la información estructural de las moléculas y comparar los resultados con los valores experimentales
![esol](https://raw.githubusercontent.com/jrmaza/machine-learning/main/ESOL_paper.png)

En esta publicación, Delaney se basó en 7 parámetros calculados a partir de la estructura molecular:
1. Constante de partición octanol agua, $clogP$
2. Peso molecular total, $MWT$
3. Número de enlaces con la capacidad de rotar $RB$
4. Proporción de átomos aromáticos en las moléculas $AP$
5. Proporción de átomos diferentes al carbono $nonCP$
6. Número de átomos donadores de enlaces de hidrógenos (H-bond donor) $DN$
7. Número de átomos aceptores de enlaces dee hidrógenos (H-bond acceptor) $AC$

Finalmente, obtubo la siguiente ecuación multiparamétrica para estimar los valores de solubilidad
\begin{equation}
Log(S_w) = 0.16 - 0.63clogP - 0.0062MWT + 0.066RB - 0.74AP
\end{equation} 

El conjunto de datos organizados en un archivo csv es proporcionado como [material suplementario](https://pubs.acs.org/doi/suppl/10.1021/ci034243x/suppl_file/ci034243xsi20040112_053635.txt) en la página de la [publicación](https://pubs.acs.org/doi/10.1021/ci034243x)



### **2.1 Obtener el conjunto de datos usando pandas**
Convenientemente, ubiqué en mi [github](https://github.com/jrmaza/machine-learning) el archivo [csv](https://raw.githubusercontent.com/jrmaza/machine-learning/main/delaney_solubility.csv) para poder utilizarlo con ```pandas```



In [ ]:
import pandas as pd #importamos librería pandas como pd
database_url = 'https://raw.githubusercontent.com/jrmaza/machine-learning/main/delaney_solubility.csv' #especificamos la URL de donde obtendremos los datos
d_sol = pd.read_csv(database_url) #le indicamos a pandas que lea el csv de la url y lo almacene en un dataframe que llamaremos d_sol (por delaney_solubility)
d_sol[:5] #mostraremos las 5 primeras entradas del dataframe
     

### **2.2 Información de las columnas del DataFrame**
El comando ```.colums``` permite apreciar la información de las columnas que conforman nuestro ```dataframe```, estos valores son referenciados por comillas simples y separados por comas. <font color='red'>Notemos que la columna **SMILES** contiene el la codificación SMILES de la molécula.</font>   


In [ ]:
d_sol.columns

Así, tenemos cuatro columnas:
* ```Compound ID```
* ```measured log(solubility:mol/L)```
* ```ESOL predicted log(solubility:mol/L)```
* ```SMILES```


### **2.3 Renombrar las columnas**
Para aligerar el tratamiento de datos, vamos a renombrar con la función ```.rename(columns = {'nombre antiguo':'nombre nuevo'}, inplace=True)```  las columnas por valores sin espacios y más sencillos:
* ```Compound ID -> CID```
* ```measured log(solubility:mol/L) -> MSOL```
* ```ESOL predicted log(solubility:mol/L) -> ESOL```
* ```SMILES = SMILES```




In [ ]:
d_sol.rename(columns = {'Compound ID':'CID'}, inplace = True)
d_sol.rename(columns = {'measured log(solubility:mol/L)':'MSOL'}, inplace = True)
d_sol.rename(columns = {'ESOL predicted log(solubility:mol/L)':'ESOL'}, inplace = True)
d_sol

### **3. RDKit y descriptores moleculares**
Los descriptores moleculares pueden ser definidos como representaciones matemáticas de las propiedades de las moléculas, son generados por algoritmos. Los valores numéricos de un descriptor molecular, son utilizados para describir cuantitativamente las propiedades físicas y químicas de las moléculas. Un ejemplo de un descriptor molecular es el logaritmo de la constante de partición octano/agua: logP, que es una representación cuantitativa de la lipofilicidad de una sustancia, es medido en condiciones experimentales como la repartición de las sustancias en diferentes fases (acuosa y n-octanol)

Con RDKit se pueden calcular descriptores, empleando el módulo ```Descriptors```

Los descriptores que podemos calcular con RDKit son:

| Descriptor | Chem.Descriptor | Sintáxis | Medida | 
| --- | --- | --- | --- |
| Peso molecular | MolWt | Descriptors.MolWt(mol) | Arroja el peso molecular con 3 cifras decimales |
| Peso molecular exacto | ExactMolWt | Descriptors.ExactMolWt(mol) | Peso molecular con 8 cifas decimales |
| Conectividad de Balaban | BalabanJ | Descriptors.BalabanJ(mol) | Índice Topológico de conectividad molecular |
| Complejidad | BertzCT | Descriptors.BertzCT(mol) | Complejidad de los enlaces y heteroátomos |
| Hibridación SP3 | FractionCSP3 | Descriptors.FractionCSP3(mol) | Arroja la fracción de átomos de carbono SP3 hibridados |
| Número de Átomos Pesados | HeavyAtomCount | Descriptors.HeavyAtomCount(mol) | Indica el número de átomos pesados diferentes al hidrógeno |
| Peso molecular  de Átomos Pesados | HeavyAtomMolWt | Descriptors.HeavyAtomMolWt(mol) | Promedio del peso molecular, ignorando hidrógenos |
| Labute | LabuteASA | Descriptors.LabuteASA(mol) | Área superficial aproximada por la ecuación de Labute |
| LogP | MolLogP | Descriptors.MolLogP(mol) | Coeficiente de partición de Wildman-Crippen |
| MR | MolMR | Descriptors.MolMR(mol) | Valor de refractividad molar, según Wildman-Crippen |
| Cargas Atómicas | MaxPartialCharge | Descriptors.MaxPartialCharge(mol) | Valor máximo de carga parcial |
|  | MinPartialCharge | Descriptors.MinPartialCharge(mol) | Valor mínimo de carga parcial |
| Conteo de átomos | NOCount | Descriptors.NOCount(mol) | Número de átomos de Nitrógeno y Oxígeno |
| Estructuras Alifáticas | NumAliphaticCarbocycles | Descriptors.NumAliphaticCarbocycles(mol) | Indica el número de carbociclos alifáticos  |
|  | NumAliphaticHeterocycles | Descriptors.NumAliphaticHeterocycles(mol) | Indica el número de heterociclos alifáticos |
|  | NumAliphaticRings | Descriptors.NumAliphaticRings(mol) | Número de anillos alifáticos |
| Estructuras Aromáticas | NumAromaticCarbocycles | Descriptors.NumAromaticCarbocycles(mol) | Indica el número de carbociclos aromáticos  |
|  | NumAromaticHeterocycles | Descriptors.NumAromaticHeterocycles(mol) | Indica el número de heterociclos aromáticos |
|  | NumAromaticRings | Descriptors.NumAromaticRings(mol) | Número de anillos aromáticos |
| Saturación | NumSaturatedCarbocycles | Descriptors.NumSaturatedCarbocycles(mol) | Número de carbociclos saturados  |
|  | NumSaturatedHeterocycles | Descriptors.NumSaturatedHeterocycles(mol) | Indica el número de heterociclos saturados |
|  | NumSaturatedRings | Descriptors.NumSaturatedRings(mol) | Número de anillos saturados |
| Aceptores de hidrógeno | NumHAcceptors | Descriptors.NumHAcceptors(mol) | Número de átomos aceptores de hidrógeno |
| Donadores de hidrógeno | NumHDonors | Descriptors.NumHDonors(mol) | Número de átomos donadores de hidrógeno |
| Heteroátomos | NumHeteroatoms | Descriptors.NumHeteroatoms(mol) | Número de átomos diferentes al Carbono e hidrógeno |
| Radicales | NumRadicalElectrons | Descriptors.NumRadicalElectrons(mol) | Número de electrones radicalarios, no indica el spin |
| Enlaces Rotables | NumRotatableBonds | Descriptors.NumRotatableBonds(mol) | Número de enlaces rotables |
| Electrones de Valencia | NumValenceElectrons | Descriptors.NumValenceElectrons(mol) | Número de electrones de Valencia |
| Anillos | RingCount | Descriptors.RingCount(mol) | Número de anillos |
| Área superficial | TPSA | Descriptors.TPSA(mol) | Área superficial polar |

### **3.1 Cargar librerías y paquetes de RDKit**


In [ ]:
import numpy as np #importamos la librería numérica numpy
import rdkit #import all RDKit
from rdkit import Chem # importamos el módulo Chem (chemistry) de RDKit
from rdkit.Chem import Descriptors # importamos el módulo Descriptors para el cálculo de descriptores moleculares
from rdkit.Chem import Draw #importamos el módulo Draw visualizar moléculas en 2D
from rdkit.Chem.Draw import IPythonConsole #permitimos a Draw insertar imágenes en el Colab
from rdkit.Chem import AllChem

### **3.2 Obtener los SMILES del DatFrame**
Extraeremos sólamente la columna de SMILES del dataframe con una iteración ```for```


In [ ]:
smiles_list = [Chem.MolFromSmiles(element) for element in d_sol.SMILES]
print(len(smiles_list))

tenemos 1144 objetos que corresponden a las entradas de la columna de nombre SMILES del DataFrame ```d_sol``` que creamos en la sección 2.1

### **3.3 Cálculo de descriptores moleculares en RDKit**

Ahora, utilizaremos RDKit y los módulos que importamos para saber cómo se calculan algunos descriptores moleculares, que utilizó Delaney para producir su modelo multiparamétrico de predicción de la solubilidad. Tomaremos una molécula al azar de la lista de smiles creada en el paso anterior.

In [ ]:
random_mol = smiles_list[411]
random_mol

#### **3.3.1 El número total de átomos de una molécula**
El sufijo ```GetNumAtoms()``` devuelte el número total de átomos de la molécula




In [ ]:
random_mol.GetNumAtoms()

#### **3.3.2 El número total de carbonos en una molécula**
Ahora crearemos un patrón (pattern) utilizando la notación ```SMARTS``` para identificar cuántos átomos en la molécula coinciden con este patrón.

In [ ]:
patron = Chem.MolFromSmarts("[C,c]") #patrón que identifica carbonos no aromáticos "C" y aromáticos "c"
numero_de_c = random_mol.GetSubstructMatches(patron)
print(len(numero_de_c))

#### **3.3.3 Peso molecular total**
La sub-rutina ```Descriptors.MolWt(**arg)``` determina el peso molecular total exacto, con muchas cifras decimales de nuestra molécula ranndom

In [ ]:
pm = Descriptors.MolWt(random_mol)
pm

#### **3.3.4 Constante de partición O/W**
La sub-rutina ```Descriptors.MolLogP(**arg)``` determina el valor teórico de la constante de partición octanol-agua

In [ ]:
clogp = Descriptors.MolLogP(random_mol)
clogp

#### **3.3.5 Enlaces rotables**
La sub-rutina ```Descriptors.NumRotatableBonds(**arg)``` determina si existen y el número de enlaces con la capacidad de rotar que tiene la molécula

In [ ]:
rotables = Descriptors.NumRotatableBonds(random_mol)
rotables

#### **3.3.6 Átomos aceptores/donadores de enlaces de hidrógeno**
Las sub-rutinas ```Descriptors.NumHAcceptors(**arg)``` y  ```Descriptors.NumHDonors(**arg)``` determinan los átomos que poseen la capacidad de aceptar y/o donar enlaces de hidrógeno

In [ ]:
donor = Descriptors.NumHDonors(random_mol)
aceptor = Descriptors.NumHAcceptors(random_mol)
print(donor, aceptor)

#### **3.3.7 Proporción de átomos aromáticos y no carbonoides**
Crearemos dos funciones que nos asistirán a determinar las proporciones aromáticas recorriiendo cada átomo de la molécula y por cada acierto incrementa un contador en 1. Finalmente, divide el número átomos aromáticos entre el número total de átomos de la molécula, resultando la proporción de átomos aromáticos. La proporción de átomos no carbonoides 

In [ ]:
def ProporcionAromatica(m):
  atomos_aromaticos = [m.GetAtomWithIdx(i).GetIsAromatic() for i in range(m.GetNumAtoms())]
  aa_count = []
  for i in atomos_aromaticos:
    if i==True:
      aa_count.append(1)
  TotAtomAromatic = sum(aa_count)
  TotAtomMol = Descriptors.HeavyAtomCount(m)
  PropArom = TotAtomAromatic/TotAtomMol
  return PropArom

def PropNonCAtomos(m):
  pattern = Chem.MolFromSmarts("[C,c]")
  CAtoms = m.GetSubstructMatches(pattern)
  TotAtomMol = Descriptors.HeavyAtomCount(m)
  NonCAtoms = TotAtomMol - len(CAtoms)
  PropNCAt = NonCAtoms/TotAtomMol
  return PropNCAt

Probaremos a continuación las funciones que acabamos de crear

In [ ]:
print("Proporción de átomos aromáticos: ",ProporcionAromatica(random_mol))
print("Proporción de átomos no carbonoides: ",PropNonCAtomos(random_mol))

#### **3.3.8 Función General para el Cálculo de todos los descriptores deseados**
La siguiente función recibe una serie de SMILES y realiza los cálculos de los descriptores moleculares anteriormente descritos, finalmente arroja una estructura ```DataFrame``` con los datos calculados


In [ ]:
def generar_descriptores(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_MolWt = Descriptors.MolWt(mol)
        desc_NumRotatableBonds = Descriptors.NumRotatableBonds(mol)
        desc_AromaticProportion = ProporcionAromatica(mol)
        desc_NonCarbonoidProportion = PropNonCAtomos(mol)
        desc_NumDonors = Descriptors.NumHDonors(mol) 
        desc_NumAceptors = Descriptors.NumHAcceptors(mol)

        row = np.array([desc_MolLogP,
                        desc_MolWt,
                        desc_NumRotatableBonds,
                        desc_AromaticProportion,
                        desc_NonCarbonoidProportion,
                        desc_NumDonors,
                        desc_NumAceptors])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["logP","PMolec","Rotable","PrAromatic","PrNonC","Donor","Acept"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

## **4. Preparación de los datos para Aprendizaje automatizado**
A continuación realizaremos una serie de operaciones que nos permitirán organizar nuestros datos para introducirlos en algoritmos de aprendizaje automatizade en ```python```

### **4.1 ```DatFrame``` con los descriptores**
Crearemos ahora un ```DataFrame``` con los descriptores escritos dentro de la función. Esta será nueestra serie de datos que utilizaremos para incorporarla en ```pycaret``` y realizar aprendizaje automatizado con nuestros datos.

In [ ]:
X = generar_descriptores(d_sol.SMILES)
X

### **4.2 Valores de solubilidad a predecir (calculados)**
De los datos originales de Delaney, extraeremos la columna con los valores de solubilidad experimentales o medidos ```MSOL```



In [ ]:
Y = d_sol.MSOL
Y

### **4.3 Verificar el tamaño de las matrices**
Debemos tener en cuenta que el tamaño de las matrices que vamos a utilizar para entrenar el modelo (los datos que están en las 7 columnas) tenga el mismo tamaño de los valores a predecir

In [ ]:
print("El tamaño de los datos de entrada es:", X.size/len(X.axes[1]))
print("El número de columnas del DataFrame es:", len(X.axes[1]))
print("Los nombres de las columnas ",X.columns)
print("El tamaño de los datos de predicción es:", float(Y.size))
if X.size/len(X.axes[1]) == float(Y.size):
  print("La estructura de datos es correcta, proceder") 
else:
  print("Estructura de datos incorrecta, revisar") 

### **4.4 Unimos las dos matrices en un solo ```DataFrame```**
Para ello utilizamos el modulo ```concat```

In [ ]:
datos = pd.concat([X,Y], axis=1)
datos

### **4.5 Escribir un csv con los datos pre-procesados**
Pandas permite guardar  ```concat```

In [ ]:
datos.to_csv('delaney_descriptores.csv', index=False, encoding='utf-8')